In [72]:
import findspark
findspark.init('/home/ubuntu/spark-2.1.1-bin-hadoop2.7')
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('operations').getOrCreate()

In [73]:
dataset1 = spark.read.csv('Datasets/traindataset.csv', inferSchema=True, header=True)
dataset2 = spark.read.csv('Datasets/testdataset.csv', inferSchema=True, header=True)
dataset1.printSchema()
dataset2.printSchema()

root
 |-- id: integer (nullable = true)
 |-- gender: string (nullable = true)
 |-- age: double (nullable = true)
 |-- hypertension: integer (nullable = true)
 |-- heart_disease: integer (nullable = true)
 |-- ever_married: string (nullable = true)
 |-- work_type: string (nullable = true)
 |-- Residence_type: string (nullable = true)
 |-- avg_glucose_level: double (nullable = true)
 |-- bmi: double (nullable = true)
 |-- smoking_status: string (nullable = true)

root
 |-- id: integer (nullable = true)
 |-- gender: string (nullable = true)
 |-- age: double (nullable = true)
 |-- hypertension: integer (nullable = true)
 |-- heart_disease: integer (nullable = true)
 |-- ever_married: string (nullable = true)
 |-- work_type: string (nullable = true)
 |-- Residence_type: string (nullable = true)
 |-- avg_glucose_level: double (nullable = true)
 |-- bmi: double (nullable = true)
 |-- smoking_status: string (nullable = true)



In [74]:
dataset1.show()
dataset2.show()

+-----+------+----+------------+-------------+------------+-------------+--------------+-----------------+----+---------------+
|   id|gender| age|hypertension|heart_disease|ever_married|    work_type|Residence_type|avg_glucose_level| bmi| smoking_status|
+-----+------+----+------------+-------------+------------+-------------+--------------+-----------------+----+---------------+
|30669|  Male| 3.0|           0|            0|          No|     children|         Rural|            95.12|18.0|           null|
|30468|  Male|58.0|           1|            0|         Yes|      Private|         Urban|            87.96|39.2|   never smoked|
|16523|Female| 8.0|           0|            0|          No|      Private|         Urban|           110.89|17.6|           null|
|56543|Female|70.0|           0|            0|         Yes|      Private|         Rural|            69.04|35.9|formerly smoked|
|46136|  Male|14.0|           0|            0|          No| Never_worked|         Rural|           161.2

In [75]:
dataset1.count()

43400

In [76]:
len(dataset1.columns)

11

In [77]:
dataset2.count()

18601

In [78]:
len(dataset2.columns)

11

In [79]:
concat = dataset1.union(dataset2)
concat.show()

+-----+------+----+------------+-------------+------------+-------------+--------------+-----------------+----+---------------+
|   id|gender| age|hypertension|heart_disease|ever_married|    work_type|Residence_type|avg_glucose_level| bmi| smoking_status|
+-----+------+----+------------+-------------+------------+-------------+--------------+-----------------+----+---------------+
|30669|  Male| 3.0|           0|            0|          No|     children|         Rural|            95.12|18.0|           null|
|30468|  Male|58.0|           1|            0|         Yes|      Private|         Urban|            87.96|39.2|   never smoked|
|16523|Female| 8.0|           0|            0|          No|      Private|         Urban|           110.89|17.6|           null|
|56543|Female|70.0|           0|            0|         Yes|      Private|         Rural|            69.04|35.9|formerly smoked|
|46136|  Male|14.0|           0|            0|          No| Never_worked|         Rural|           161.2

In [80]:
concat.count()

62001

In [81]:
len(concat.columns)

11

In [82]:
concat.groupby('gender').count().show()

+------+-----+
|gender|count|
+------+-----+
|Female|36622|
| Other|   13|
|  Male|25366|
+------+-----+



In [83]:
concat.groupby('hypertension').count().show()

+------------+-----+
|hypertension|count|
+------------+-----+
|           1| 5794|
|           0|56207|
+------------+-----+



In [84]:
concat.groupby('work_type').count().show()

+-------------+-----+
|    work_type|count|
+-------------+-----+
| Never_worked|  252|
|Self-employed| 9654|
|      Private|35584|
|     children| 8769|
|     Govt_job| 7742|
+-------------+-----+



In [85]:
concat.groupby('residence_type').count().show()

+--------------+-----+
|residence_type|count|
+--------------+-----+
|         Urban|31066|
|         Rural|30935|
+--------------+-----+



In [86]:
concat.groupby('ever_married').count().show()

+------------+-----+
|ever_married|count|
+------------+-----+
|          No|22124|
|         Yes|39877|
+------------+-----+



In [87]:
concat.groupby('smoking_status').count().show()

+---------------+-----+
| smoking_status|count|
+---------------+-----+
|           null|19043|
|         smokes| 9319|
|   never smoked|22886|
|formerly smoked|10753|
+---------------+-----+



In [88]:
concat.groupby('heart_disease').count().show()

+-------------+-----+
|heart_disease|count|
+-------------+-----+
|            1| 2956|
|            0|59045|
+-------------+-----+



In [89]:
concat.describe().show()

+-------+-----------------+------+------------------+-------------------+-------------------+------------+---------+--------------+-----------------+-----------------+---------------+
|summary|               id|gender|               age|       hypertension|      heart_disease|ever_married|work_type|Residence_type|avg_glucose_level|              bmi| smoking_status|
+-------+-----------------+------+------------------+-------------------+-------------------+------------+---------+--------------+-----------------+-----------------+---------------+
|  count|            62001| 62001|             62001|              62001|              62001|       62001|    62001|         62001|            62001|            59948|          42958|
|   mean|36452.51481427719|  null|42.169475331042975|0.09345010564345736|0.04767665037660683|        null|     null|          null| 104.453831712391| 28.5870988189766|           null|
| stddev|21067.15594488889|  null|22.522099535371346| 0.2910645114143387|0.21308

In [90]:
concat.crosstab('gender', 'heart_disease').show()

+--------------------+-----+----+
|gender_heart_disease|    0|   1|
+--------------------+-----+----+
|               Other|   13|   0|
|                Male|23606|1760|
|              Female|35426|1196|
+--------------------+-----+----+



In [91]:
concat.crosstab('smoking_status', 'heart_disease').show()

+----------------------------+-----+---+
|smoking_status_heart_disease|    0|  1|
+----------------------------+-----+---+
|             formerly smoked| 9818|935|
|                      smokes| 8713|606|
|                never smoked|22024|862|
|                        null|18490|553|
+----------------------------+-----+---+



In [92]:
concat.crosstab('ever_married', 'heart_disease').show()

+--------------------------+-----+----+
|ever_married_heart_disease|    0|   1|
+--------------------------+-----+----+
|                        No|21893| 231|
|                       Yes|37152|2725|
+--------------------------+-----+----+



In [93]:
concat.crosstab('work_type', 'heart_disease').show()

+-----------------------+-----+----+
|work_type_heart_disease|    0|   1|
+-----------------------+-----+----+
|               children| 8765|   4|
|           Never_worked|  251|   1|
|          Self-employed| 8718| 936|
|                Private|33946|1638|
|               Govt_job| 7365| 377|
+-----------------------+-----+----+



In [94]:
concat.crosstab('residence_type', 'heart_disease').show()

+----------------------------+-----+----+
|residence_type_heart_disease|    0|   1|
+----------------------------+-----+----+
|                       Rural|29441|1494|
|                       Urban|29604|1462|
+----------------------------+-----+----+



In [95]:
concat.crosstab('hypertension', 'heart_disease').show()

+--------------------------+-----+----+
|hypertension_heart_disease|    0|   1|
+--------------------------+-----+----+
|                         1| 5040| 754|
|                         0|54005|2202|
+--------------------------+-----+----+



In [151]:
concat.drop('id').columns

['gender',
 'age',
 'hypertension',
 'heart_disease',
 'ever_married',
 'work_type',
 'Residence_type',
 'avg_glucose_level',
 'bmi',
 'smoking_status']

In [107]:
from pyspark.sql.functions import col,sum

In [136]:
concat.select(*(sum(col(c).isNull().cast("int")).alias(c) for c in concat.columns)).show()

+---+------+---+------------+-------------+------------+---------+--------------+-----------------+----+--------------+
| id|gender|age|hypertension|heart_disease|ever_married|work_type|Residence_type|avg_glucose_level| bmi|smoking_status|
+---+------+---+------------+-------------+------------+---------+--------------+-----------------+----+--------------+
|  0|     0|  0|           0|            0|           0|        0|             0|                0|2053|         19043|
+---+------+---+------------+-------------+------------+---------+--------------+-----------------+----+--------------+



In [154]:
clean1 = concat.select(['gender','age','hypertension','heart_disease','ever_married','work_type','Residence_type','avg_glucose_level','bmi','smoking_status'])

In [155]:
clean = clean1.dropna()

In [157]:
clean.show()

+------+----+------------+-------------+------------+-------------+--------------+-----------------+----+---------------+
|gender| age|hypertension|heart_disease|ever_married|    work_type|Residence_type|avg_glucose_level| bmi| smoking_status|
+------+----+------------+-------------+------------+-------------+--------------+-----------------+----+---------------+
|  Male|58.0|           1|            0|         Yes|      Private|         Urban|            87.96|39.2|   never smoked|
|Female|70.0|           0|            0|         Yes|      Private|         Rural|            69.04|35.9|formerly smoked|
|Female|52.0|           0|            0|         Yes|      Private|         Urban|            77.59|17.7|formerly smoked|
|Female|75.0|           0|            1|         Yes|Self-employed|         Rural|           243.53|27.0|   never smoked|
|Female|32.0|           0|            0|         Yes|      Private|         Rural|            77.67|32.3|         smokes|
|Female|74.0|           

In [111]:
concat.describe('age').show()

+-------+------------------+
|summary|               age|
+-------+------------------+
|  count|             62001|
|   mean|42.169475331042975|
| stddev|22.522099535371346|
|    min|              0.08|
|    max|              82.0|
+-------+------------------+



In [112]:
concat.describe('avg_glucose_level').show()

+-------+-----------------+
|summary|avg_glucose_level|
+-------+-----------------+
|  count|            62001|
|   mean| 104.453831712391|
| stddev| 42.9605371949353|
|    min|             55.0|
|    max|           291.05|
+-------+-----------------+



In [113]:
concat.describe('bmi').show()

+-------+-----------------+
|summary|              bmi|
+-------+-----------------+
|  count|            59948|
|   mean| 28.5870988189766|
| stddev|7.766122190731495|
|    min|             10.1|
|    max|             97.6|
+-------+-----------------+



In [158]:
clean.count()

41495

In [141]:
from pyspark.ml.regression import LinearRegression

In [119]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [120]:
assembler = VectorAssembler(
    inputCols=['age','hypertension','bmi','avg_glucose_level'],
    outputCol='features')

In [122]:
output = assembler.transform(clean)

In [123]:
output.printSchema()
output.head(1)

root
 |-- id: integer (nullable = true)
 |-- gender: string (nullable = true)
 |-- age: double (nullable = true)
 |-- hypertension: integer (nullable = true)
 |-- heart_disease: integer (nullable = true)
 |-- ever_married: string (nullable = true)
 |-- work_type: string (nullable = true)
 |-- Residence_type: string (nullable = true)
 |-- avg_glucose_level: double (nullable = true)
 |-- bmi: double (nullable = true)
 |-- smoking_status: string (nullable = true)
 |-- features: vector (nullable = true)



[Row(id=30468, gender='Male', age=58.0, hypertension=1, heart_disease=0, ever_married='Yes', work_type='Private', Residence_type='Urban', avg_glucose_level=87.96, bmi=39.2, smoking_status='never smoked', features=DenseVector([58.0, 1.0, 39.2, 87.96]))]

In [124]:
final_data = output.select("features",'heart_disease')
final_data.show()

+--------------------+-------------+
|            features|heart_disease|
+--------------------+-------------+
|[58.0,1.0,39.2,87...|            0|
|[70.0,0.0,35.9,69...|            0|
|[52.0,0.0,17.7,77...|            0|
|[75.0,0.0,27.0,24...|            1|
|[32.0,0.0,32.3,77...|            0|
|[74.0,1.0,54.6,20...|            0|
|[79.0,0.0,22.0,57...|            1|
|[37.0,0.0,39.4,16...|            0|
|[37.0,0.0,26.1,73.5]|            0|
|[40.0,0.0,42.4,95...|            0|
|[35.0,0.0,33.0,85...|            0|
|[20.0,0.0,19.7,84...|            0|
|[42.0,0.0,22.5,82...|            0|
|[44.0,0.0,24.6,57...|            0|
|[79.0,0.0,25.2,67...|            1|
|[57.0,1.0,60.9,12...|            0|
|[49.0,0.0,31.5,60...|            0|
|[71.0,0.0,27.3,19...|            0|
|[59.0,0.0,23.7,10...|            0|
|[25.0,0.0,24.5,60...|            0|
+--------------------+-------------+
only showing top 20 rows



In [125]:
train_data,test_data = final_data.randomSplit([0.7,0.3])

In [126]:
lr = LinearRegression(labelCol='heart_disease')

In [127]:
lrModel = lr.fit(train_data)

In [128]:
print("Coefficients: {} Intercept: {}".format(lrModel.coefficients,lrModel.intercept))

Coefficients: [0.0025172168189739803,0.04438122542377693,-0.0006548965133557565,0.0003657486798979235] Intercept: -0.09294494482725153


In [129]:
test_results = lrModel.evaluate(test_data)

In [130]:
test_results.residuals.show()
print("RSME: {}".format(test_results.rootMeanSquaredError))

+--------------------+
|           residuals|
+--------------------+
| 0.04670129461216291|
|0.034920529632650785|
|0.054831542655247686|
| 0.05731794345646147|
| 0.03971411438192836|
|0.049931268208338514|
|0.053174768776940995|
|0.047308159951378305|
| 0.04661082168224989|
| 0.05510681840523261|
|0.047277035767048635|
|0.055304390334008385|
|0.030762653912857714|
| 0.04795477379962884|
|0.007982100573584783|
| 0.02915184054241507|
| 0.05595217825010465|
|  0.0454863829629948|
|0.039728463408263345|
|0.054514647203398235|
+--------------------+
only showing top 20 rows

RSME: 0.2181333535852703


In [131]:
print("R2: {}".format(test_results.r2))

R2: 0.07204817318068812


In [132]:
from pyspark.ml.classification import LogisticRegression

In [159]:
from pyspark.ml.feature import (VectorAssembler,VectorIndexer,
                                OneHotEncoder,StringIndexer)

In [160]:
gender_indexer = StringIndexer(inputCol='gender',outputCol='GenderIndex')
gender_encoder = OneHotEncoder(inputCol='GenderIndex',outputCol='GenderVec')

In [161]:
work_indexer = StringIndexer(inputCol='work_type',outputCol='WorkTypeIndex')
work_encoder = OneHotEncoder(inputCol='WorkTypeIndex',outputCol='WorkTypeVec')

In [162]:
married_indexer = StringIndexer(inputCol='ever_married',outputCol='EverMarriedIndex')
married_encoder = OneHotEncoder(inputCol='EverMarriedIndex',outputCol='EverMarriedVec')

In [163]:
residence_indexer = StringIndexer(inputCol='Residence_type',outputCol='ResidenceTypeIndex')
residence_encoder = OneHotEncoder(inputCol='ResidenceTypeIndex',outputCol='ResidenceTypeVec')

In [164]:
smoking_indexer = StringIndexer(inputCol='smoking_status',outputCol='SmokingStatusIndex')
smoking_encoder = OneHotEncoder(inputCol='SmokingStatusIndex',outputCol='SmokingStatusVec')

In [165]:
assembler = VectorAssembler(inputCols=['age',
 'GenderVec',
 'bmi',
 'avg_glucose_level',
 'hypertension',
 'WorkTypeVec',
 'EverMarriedVec','ResidenceTypeVec', 'SmokingStatusVec'],outputCol='features')

In [169]:
from pyspark.ml import Pipeline

In [180]:
log_reg_heart = LogisticRegression(featuresCol='features',labelCol='heart_disease')

In [181]:
pipeline = Pipeline(stages=[gender_indexer,work_indexer,married_indexer,residence_indexer,smoking_indexer,
                           gender_encoder,work_encoder,married_encoder,residence_encoder,smoking_encoder,
                           assembler,log_reg_heart])

In [182]:
train_heart_data, test_heart_data = clean.randomSplit([0.7,.3])

In [183]:
fit_model = pipeline.fit(train_heart_data)

In [184]:
results = fit_model.transform(test_heart_data)

In [185]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

my_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction',
                                       labelCol='heart_disease')

In [186]:
results.select('heart_disease','prediction').show()

+-------------+----------+
|heart_disease|prediction|
+-------------+----------+
|            0|       0.0|
|            0|       0.0|
|            0|       0.0|
|            0|       0.0|
|            0|       0.0|
|            0|       0.0|
|            0|       0.0|
|            0|       0.0|
|            0|       0.0|
|            0|       0.0|
|            0|       0.0|
|            0|       0.0|
|            0|       0.0|
|            0|       0.0|
|            0|       0.0|
|            0|       0.0|
|            0|       0.0|
|            0|       0.0|
|            0|       0.0|
|            0|       0.0|
+-------------+----------+
only showing top 20 rows



In [187]:
AUC = my_eval.evaluate(results)

AUC

0.5038200699809988

In [188]:
totalResults = results.select('heart_disease','prediction')

correctResults = totalResults.filter(totalResults['heart_disease'] == totalResults['prediction'])

countTR = totalResults.count()
print("Correct: " + str(countTR))

countTC = correctResults.count()
print("Total Correct: " + str(countTC)) 

Correct: 12402
Total Correct: 11782


In [189]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [220]:
assembler = VectorAssembler(
  inputCols=['age',
             'bmi',
             'avg_glucose_level',
             'hypertension'],
              outputCol="features1")

In [221]:
output = assembler.transform(clean)

In [222]:
from pyspark.ml.feature import StringIndexer

In [223]:
indexer = StringIndexer(inputCol="heart_disease", outputCol="HeartDiseaseIndex")
output_fixed = indexer.fit(output).transform(output)

In [224]:
final_data = output_fixed.select("features1",'HeartDiseaseIndex')

In [229]:
train1_data,test1_data = final_data.randomSplit([0.7,0.3])

In [230]:
from pyspark.ml.classification import DecisionTreeClassifier,GBTClassifier,RandomForestClassifier
from pyspark.ml import Pipeline

In [231]:
dtc = DecisionTreeClassifier(labelCol='HeartDiseaseIndex',featuresCol='features1')
rfc = RandomForestClassifier(labelCol='HeartDiseaseIndex',featuresCol='features1')
gbt = GBTClassifier(labelCol='HeartDiseaseIndex',featuresCol='features1')

In [232]:
dtc_model = dtc.fit(train1_data)
rfc_model = rfc.fit(train1_data)
gbt_model = gbt.fit(train1_data)

In [233]:
dtc_predictions = dtc_model.transform(test1_data)
rfc_predictions = rfc_model.transform(test1_data)
gbt_predictions = gbt_model.transform(test1_data)

In [234]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
my_binary_eval = BinaryClassificationEvaluator(labelCol = 'HeartDiseaseIndex')

In [235]:
print("DTC")
print(my_binary_eval.evaluate(dtc_predictions))

print("RFC")
print(my_binary_eval.evaluate(rfc_predictions))

my_binary_gbt_eval = BinaryClassificationEvaluator(labelCol='HeartDiseaseIndex', rawPredictionCol='prediction')
print("GBT")
print(my_binary_gbt_eval.evaluate(gbt_predictions))

DTC
0.7651795296279124
RFC
0.8508056901569699
GBT
0.49995755157483657


In [236]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [237]:
acc_evaluator = MulticlassClassificationEvaluator(labelCol="HeartDiseaseIndex", predictionCol="prediction", metricName="accuracy")

In [238]:
dtc_acc = acc_evaluator.evaluate(dtc_predictions)
rfc_acc = acc_evaluator.evaluate(rfc_predictions)
gbt_acc = acc_evaluator.evaluate(gbt_predictions)

In [239]:
print("Here are the results!")
print('-'*40)
print('A single decision tree has an accuracy of: {0:2.2f}%'.format(dtc_acc*100))
print('-'*40)
print('A random forest ensemble has an accuracy of: {0:2.2f}%'.format(rfc_acc*100))
print('-'*40)
print('An ensemble using GBT has an accuracy of: {0:2.2f}%'.format(gbt_acc*100))

Here are the results!
----------------------------------------
A single decision tree has an accuracy of: 94.97%
----------------------------------------
A random forest ensemble has an accuracy of: 94.97%
----------------------------------------
An ensemble using GBT has an accuracy of: 94.96%


In [240]:
rfc_predictions.show()

+--------------------+-----------------+--------------------+--------------------+----------+
|           features1|HeartDiseaseIndex|       rawPrediction|         probability|prediction|
+--------------------+-----------------+--------------------+--------------------+----------+
|[10.0,15.7,139.92...|              0.0|[19.8629104248238...|[0.99314552124119...|       0.0|
|[10.0,15.9,87.52,...|              0.0|[19.8894023116055...|[0.99447011558027...|       0.0|
|[10.0,16.5,106.26...|              0.0|[19.8959929898701...|[0.99479964949350...|       0.0|
|[10.0,17.1,112.21...|              0.0|[19.8959929898701...|[0.99479964949350...|       0.0|
|[10.0,17.4,91.01,...|              0.0|[19.9070161025563...|[0.99535080512781...|       0.0|
|[10.0,17.6,94.91,...|              0.0|[19.8856243188621...|[0.99428121594310...|       0.0|
|[10.0,17.6,107.48...|              0.0|[19.8959929898701...|[0.99479964949350...|       0.0|
|[10.0,18.0,109.17...|              0.0|[19.8959929898701...